### Sentence embeddings used - https://github.com/UKPLab/sentence-transformers

## To create the environment, I ran : 
 - conda create --name py37 python=3.7

 - conda activate py37

 - pip install -U sentence-transformers

 - conda install -c anaconda pandas

 - conda install -c anaconda openpyxl           
 
  (*installed openpyxl because pandas gave an error and asked for that package*)

In [1]:
import pandas as pd

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

/home/idwivedi/anaconda3/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataframe_bt = pd.read_csv('breakthrough-therapies.csv')
dataframe_bt

,Trade Name (Agent),Sponsor,Date of BT Designation Disclosure,Approval Date,Indication,Category,FDA Status
0,Imcivree (setmelanotide),"Rhythm Pharmaceutical, Inc.",1/7/2016,6/16/2022,For chronic weight management in adult and ped...,Other,FDA Approval
1,Gedatolisib,Celcuity Inc.,7/18/2022,NaN,Treatment of patients with HR+/HER2- metastati...,Cancer,Designation Granted
2,Dupixent (dupilumab),Regeneron Pharmaceuticals Inc.,10/1/2016,6/7/2022,Treatment of adult and pediatric patients ages...,Other,FDA Approval
3,VIJOICE (alpelisib),Novartis Pharmaceuticals Corp.,11/13/2019,4/5/2022,Treatment of adult and pediatric patients 2 ye...,Cancer,FDA Approval
4,Apretude (Cabotegravir),ViiV Healthcare,11/17/2020,12/20/2021,HIV-1 pre-exposure prophylaxis (PrEP) to reduc...,Infectious Disease,FDA Approval
...,...,...,...,...,...,...,...
463,Koselugo (Selumetinib),"AstraZeneca Plc and Merck & Co., Inc.",4/01/2019,4/10/2020,Treatment of pediatric patients 2 years of age...,Rare Inherited Disorders,FDA Approval
464,TUKYSA (tucatinib),SEATTLE GENETICS INC,2019,4/17/2020,In combination with trastuzumab and capecitabi...,Cancer,FDA Approval
465,Enspryng (satralizumab-mwge),"Genentech, Inc.",12/20/2018,8/14/2020,Subcutaneous treatment for adults living with ...,Rare Inherited Disorders,FDA Approval
466,Artesunate,AMIVAS LLC,2020,5/26/2020,Initial treatment of severe malaria in adult a...,Other,FDA Approval


In [4]:
Indication_list = []
for i in range(468):
    Indication_list.append(dataframe_bt.iloc[i,4])

In [5]:
sentence_embeddings_il = model.encode(Indication_list)

In [6]:
sentence_embeddings_il.shape

(468, 768)

In [7]:
dataframe_sj = pd.read_excel('Section111ValidICD10-Jan2022.xlsx')
dataframe_sj

,CODE,SHORT DESCRIPTION (VALID ICD-10 FY2022),LONG DESCRIPTION (VALID ICD-10 FY2022),NF EXCL
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol...","Cholera due to Vibrio cholerae 01, biovar chol...",NaN
1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor","Cholera due to Vibrio cholerae 01, biovar eltor",NaN
2,A009,"Cholera, unspecified","Cholera, unspecified",NaN
3,A0100,"Typhoid fever, unspecified","Typhoid fever, unspecified",NaN
4,A0101,Typhoid meningitis,Typhoid meningitis,NaN
...,...,...,...,...
71731,Y990,Civilian activity done for income or pay,Civilian activity done for income or pay,Y
71732,Y991,Military activity,Military activity,Y
71733,Y992,Volunteer activity,Volunteer activity,Y
71734,Y998,Other external cause status,Other external cause status,Y


In [8]:
code2des = {}

ld_list = []
for i in range(71736):
    ld = dataframe_sj.iloc[i,2].strip()
    ld_list.append(ld)

In [9]:
sentence_embeddings_ld = model.encode(ld_list)

In [10]:
for i in range(71736):
    code = dataframe_sj.iloc[i,0].strip()
    ld = dataframe_sj.iloc[i,2].strip()
    code2des[code] = (ld,sentence_embeddings_ld[i])

In [11]:
import numpy as np
from numpy.linalg import norm

def sim(a,b):
    return np.dot(a, b)/(norm(a)*norm(b))

In [12]:
matched_code_list = []
matched_description = []
for idx, i in enumerate(sentence_embeddings_il):
    code_dict ={}
    print (f'{idx} of {sentence_embeddings_il.shape[0]} finished')
    for code in code2des.keys():
        code_dict[code] = sim(code2des[code][1],i)
    matched_code = max(code_dict, key=code_dict.get)
    matched_code_list.append(matched_code)
    matched_description.append(code2des[matched_code][0])

0 of 468 finished
1 of 468 finished
2 of 468 finished
3 of 468 finished
4 of 468 finished
5 of 468 finished
6 of 468 finished
7 of 468 finished
8 of 468 finished
9 of 468 finished
10 of 468 finished
11 of 468 finished
12 of 468 finished
13 of 468 finished
14 of 468 finished
15 of 468 finished
16 of 468 finished
17 of 468 finished
18 of 468 finished
19 of 468 finished
20 of 468 finished
21 of 468 finished
22 of 468 finished
23 of 468 finished
24 of 468 finished
25 of 468 finished
26 of 468 finished
27 of 468 finished
28 of 468 finished
29 of 468 finished
30 of 468 finished
31 of 468 finished
32 of 468 finished
33 of 468 finished
34 of 468 finished
35 of 468 finished
36 of 468 finished
37 of 468 finished
38 of 468 finished
39 of 468 finished
40 of 468 finished
41 of 468 finished
42 of 468 finished
43 of 468 finished
44 of 468 finished
45 of 468 finished
46 of 468 finished
47 of 468 finished
48 of 468 finished
49 of 468 finished
50 of 468 finished
51 of 468 finished
52 of 468 finished
53 

416 of 468 finished
417 of 468 finished
418 of 468 finished
419 of 468 finished
420 of 468 finished
421 of 468 finished
422 of 468 finished
423 of 468 finished
424 of 468 finished
425 of 468 finished
426 of 468 finished
427 of 468 finished
428 of 468 finished
429 of 468 finished
430 of 468 finished
431 of 468 finished
432 of 468 finished
433 of 468 finished
434 of 468 finished
435 of 468 finished
436 of 468 finished
437 of 468 finished
438 of 468 finished
439 of 468 finished
440 of 468 finished
441 of 468 finished
442 of 468 finished
443 of 468 finished
444 of 468 finished
445 of 468 finished
446 of 468 finished
447 of 468 finished
448 of 468 finished
449 of 468 finished
450 of 468 finished
451 of 468 finished
452 of 468 finished
453 of 468 finished
454 of 468 finished
455 of 468 finished
456 of 468 finished
457 of 468 finished
458 of 468 finished
459 of 468 finished
460 of 468 finished
461 of 468 finished
462 of 468 finished
463 of 468 finished
464 of 468 finished
465 of 468 finished


In [13]:
dataframe_bt['Matched Code'] = matched_code_list

In [14]:
dataframe_bt['Matched Indication'] = matched_description

In [15]:
dataframe_bt.to_excel("output.xlsx")  

In [16]:
dataframe_bt

,Trade Name (Agent),Sponsor,Date of BT Designation Disclosure,Approval Date,Indication,Category,FDA Status,Matched Code,Matched Indication
0,Imcivree (setmelanotide),"Rhythm Pharmaceutical, Inc.",1/7/2016,6/16/2022,For chronic weight management in adult and ped...,Other,FDA Approval,R6332,"Pediatric feeding disorder, chronic"
1,Gedatolisib,Celcuity Inc.,7/18/2022,NaN,Treatment of patients with HR+/HER2- metastati...,Cancer,Designation Granted,C847A,"Anaplastic large cell lymphoma, ALK-negative, ..."
2,Dupixent (dupilumab),Regeneron Pharmaceuticals Inc.,10/1/2016,6/7/2022,Treatment of adult and pediatric patients ages...,Other,FDA Approval,L2083,Infantile (acute) (chronic) eczema
3,VIJOICE (alpelisib),Novartis Pharmaceuticals Corp.,11/13/2019,4/5/2022,Treatment of adult and pediatric patients 2 ye...,Cancer,FDA Approval,Q873,Congenital malformation syndromes involving ea...
4,Apretude (Cabotegravir),ViiV Healthcare,11/17/2020,12/20/2021,HIV-1 pre-exposure prophylaxis (PrEP) to reduc...,Infectious Disease,FDA Approval,B20,Human immunodeficiency virus [HIV] disease
...,...,...,...,...,...,...,...,...,...
463,Koselugo (Selumetinib),"AstraZeneca Plc and Merck & Co., Inc.",4/01/2019,4/10/2020,Treatment of pediatric patients 2 years of age...,Rare Inherited Disorders,FDA Approval,Q8501,"Neurofibromatosis, type 1"
464,TUKYSA (tucatinib),SEATTLE GENETICS INC,2019,4/17/2020,In combination with trastuzumab and capecitabi...,Cancer,FDA Approval,C50011,"Malignant neoplasm of nipple and areola, right..."
465,Enspryng (satralizumab-mwge),"Genentech, Inc.",12/20/2018,8/14/2020,Subcutaneous treatment for adults living with ...,Rare Inherited Disorders,FDA Approval,G360,Neuromyelitis optica [Devic]
466,Artesunate,AMIVAS LLC,2020,5/26/2020,Initial treatment of severe malaria in adult a...,Other,FDA Approval,B528,Plasmodium malariae malaria with other complic...
